In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter

In [ ]:
#tutti i gruppi di test
group1 = ["1-" + str(i) for i in range(1, 11)]
group2 = ["2-" + str(i) for i in range(1, 3)]
group3 = ["3-" + str(i) for i in range(1, 4)]
group4 = ["4-" + str(i) for i in range(1, 11)]

In [ ]:
#tests contenente tutti i gruppi, types da scegliere in base ai test
tests = [group1, group2, group3, group4]
types = ["test0", "test1", "test2", "test3", "test4", "test5", "test6", "test7", "test8", "test9"]

In [ ]:
#dir dei log: percorso base 
log_dir = os.path.join(os.getcwd(), "testrun-alone")

In [ ]:
#funzione load restituisce dataframe
def load_data(file):
    # Inizializzazione dataframe con colonne di interesse
    records = pd.DataFrame()
    try:
        file_path=log_dir + '/' + types[j] + '/' + tests[l][k] + '/' + file
        # Leggo file
        records = pd.read_csv(file_path, sep='\\s+', usecols=[2,3,6,8], header=0, names=["RunT", "Period", "rStart", "cDur"])
    except FileNotFoundError:
        print("File non esiste:", file_path)
    except pd.errors.EmptyDataError:
        print("File vuoto:", file_path)
    
    return records


In [ ]:
#faccio parsing dei valori per ogni sottocartella test*/*-*
#creo tanti dataframe distinti per ogni cartella dei test

types = ["test0", "test1", "test2", "test3", "test4", "test5", "test6", "test7", "test8", "test9", "test10", "test11"]
for l in range(0, len(tests)): # da 0 a 3
    for k in range(0, len(tests[l])): # lunghezza di ogni singolo test
        testgroup = []
        pcount = []
        for j in range(0, len(types)): # per tutti i test in esame
            print (types[j] + '/' + tests[l][k])
            frames = pd.DataFrame()
            diff=0
            for f in os.listdir(log_dir + '/' + types[j] + '/' + tests[l][k]):
                if f.endswith(".log"):
                    records = load_data(f)
                    init_count = len(records)
                    records = records[~(records["RunT"] > records["cDur"] * 1.5)] #mi prendo solo i records senza overrun
                    final_count = len(records)
                    diff = diff + (init_count - final_count) #pcount totali per sottocartella (esempio somma pcount di ogni container per 1-8)
                    frames = pd.concat([frames, records]) #dataframe con tutti i container per sottocartella (esempio tutti 8 container in 1-8)

            num_entries = len(frames)
            overrun_per=round((diff/num_entries)*100) #percentuale di valori overrun su entries totali
            pcount.append(overrun_per) #tanti valori quanti i test svolti       
            testgroup.append(frames['RunT'].to_numpy()) #tanti valori quanti i test svolti 
   
       # print(pcount)
        plt.figure(figsize=(10, 6))
        box = plt.boxplot(testgroup, labels=['0', '1', '2', '3', '4', '5', '6','7', '8', '9', '10','11'], vert=True, showfliers=False, showmeans=True)
        
        list_y = []  
        max_y=-np.inf # Inizializzazione del valore minimo

        for b in box['whiskers']:
            y = b.get_ydata()  # coordinate y del box
            max_y = max(max_y, max(y))  # Trova il valore minimo delle y
        
        for b in box['boxes']:
            list_y.append(max_y+5)
        
        #print(list_y)
        x = range(1, len(testgroup) + 1)
        #print(x)
        print(pcount)
        plt.scatter(x, list_y, s=[v for v in pcount], c='blue', alpha=0.6)
        
        # Aggiungi le label con i valori di pcount sopra i punti scatter
        for i, txt in enumerate(pcount):
            plt.text(x[i], list_y[i], f'{txt}%', ha='center', va='bottom')

        plt.title('Confronto configurazioni kernel: batch ' + str(l+1) + ' con ' + str(k+1)+ ' container di carico')
        plt.xlabel('Num configurazione kernel')
        plt.ylabel('Runtime')
        plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%d us'))
        #filename = 'Test_' + tests[l][k].replace(" ", "_") + '.jpg'
        #plt.savefig(filename)
        legend_objs = [ plt.Line2D([0], [0], color='orange', lw=2, label='Mediana'),
               plt.Line2D([0], [0], marker='^', color='green', label='Media', markersize=5, linestyle='None'),
               plt.Line2D([0], [0], marker='o', color='blue', label='Percentuale di overrun', markersize=5, linestyle='None'),
               plt.Line2D([0], [0], lw=0, label='0-codesys'),
               plt.Line2D([0], [0], lw=0, label='1-nosmt'),
               plt.Line2D([0], [0], lw=0, label='2-nosmt+rcu_nocbs'),
               plt.Line2D([0], [0], lw=0, label='3-nosmt+rcu_nocbs+rcu_nocb_poll'),
               plt.Line2D([0], [0], lw=0, label='4-nosmt+irq_affinity'),
               plt.Line2D([0], [0], lw=0, label='5-nosmt+irq_affinity+rcu_nocbs'),
               plt.Line2D([0], [0], lw=0, label='6-nosmt+irqaffinity+rcu_nocb_poll+rcu_nocbs'),
               plt.Line2D([0], [0], lw=0, label='7-nosmt+irqaffinity+rcu_nocb_poll+rcu_nocbs+skew_tick'),
               plt.Line2D([0], [0], lw=0, label='8-nosmt+irqaffinity+rcu_nocb_poll+rcu_nocbs+nosoftlockup+tsc=nowatchdog'),
               plt.Line2D([0], [0], lw=0, label='9-nosmt+irqaffinity+rcu_nocb_poll+rcu_nocbs+skew_tick+nosoftlockup+tsc=nowatchdog'),
               plt.Line2D([0], [0], lw=0, label='10-nosmt+irqaffinity+rcu_nocbs+skew_tick+nosoftlockup+tsc=nowatchdog'),
               plt.Line2D([0], [0], lw=0, label='11-nosmt+irqaffinity+rcu_nocbs+skew_tick'),]

        plt.legend(handles=legend_objs, loc='center left', bbox_to_anchor=(1, 0.5))
       
        plt.show()
         

In [ ]:
#funzione load restituisce dataframe
def load_data3(file):
    # Inizializzazione dataframe con colonne di interesse
    records = pd.DataFrame()
    try:
        file_path=log_dir + '/' + types[j] + '/' + tests[2][k] + '/' + file
        # Leggo file
        records = pd.read_csv(file_path, sep='\\s+', usecols=[2,3,6,8], header=0, names=["RunT", "Period", "rStart", "cDur"])
    except FileNotFoundError:
        print("File non esiste:", file_path)
    except pd.errors.EmptyDataError:
        print("File vuoto:", file_path)
    
    return records

In [ ]:
#un boxplot per ogni container 3-1/3-2/ 3-3
#un grafico per ogni test con 2 boxplot per i due container in 3-2
#un grafico per ogni test con 3 boxplot per i due container in 3-3
types = ["test1", "test2", "test3", "test4", "test5", "test6", "test7", "test8", "test9"]
for k in range(1, len(tests[2])): # lunghezza del gruppo 3 ossia 3
    for j in range(0, len(types)): # per tutti i test in esame
        print (types[j] + '/' + tests[2][k])
        testgroup = []
        pcount = []
        for f in os.listdir(log_dir + '/' + types[j] + '/' + tests[2][k]):
            if f.endswith(".log"):
                print(f)
                records = load_data3(f)
                init_count = len(records)
                records = records[~(records["RunT"] > records["cDur"] * 1.5)] #mi prendo solo i records senza overrun
                final_count = len(records)
                diff = init_count - final_count #pcount totali per ogni container
                testgroup.append(records['RunT'].to_numpy()) #tanti valori quanti i container presenti  
                pcount.append(diff) #tanti valori quanti i container presenti   
                
        print(testgroup)
        print(pcount)

        plt.figure(figsize=(10, 6))
        box = plt.boxplot(testgroup, vert=True, showfliers=False, showmeans=True)
    
        list_y = []  
        max_y=-np.inf # Inizializzazione del valore minimo

        for b in box['whiskers']:
            y = b.get_ydata()  # coordinate y del box
            max_y = max(max_y, max(y))  # Trova il valore minimo delle y
        
        for b in box['boxes']:
            list_y.append(max_y+5)
        
        x = range(1, len(testgroup) + 1)
      
        plt.scatter(x, list_y, s=[v/100 for v in pcount], c='blue', alpha=0.6)
        plt.title('Boxplot di ' + types[j] + '/' + tests[2][k])
        plt.xlabel('Numero container')
        plt.ylabel('Runtime')
        plt.show()

In [ ]:
# Funzione per estrarre la parte dell'indice che ci interessa
def get_test_id(index):
    return index.split('/')[1]

# Gruppo di indici
grouped_indices = result.groupby(get_test_id).groups
#print (grouped_indices)
# Iterare attraverso i gruppi e stampare le coppie
for key, indices in grouped_indices.items():
    if len(indices) > 1:
        a = result.loc[indices].plot(kind='bar', y=['runMdn','runAvg','runStD'], figsize=(10, 6))
        for barra in a.patches:
            altezza = barra.get_height()
            a.text(barra.get_x()+barra.get_width()/2, altezza, f'{altezza:.2f}', ha='center', va='bottom')
        plt.xlabel('Test Number')  # Etichetta sull'asse x
        plt.ylabel('Value')  # Etichetta sull'asse y
        plt.tight_layout()
        plt.show()
       # print(f"Group: {key}")
        #print(result.loc[indices])
        #print("---")


In [ ]:
def get_test_id(index):
    return index.split('/')[0]

# Gruppo di indici
grouped_indices = result.groupby(get_test_id).groups
#print(grouped_indices)
# Iterare attraverso i gruppi e stampare le coppie
for key, indices in grouped_indices.items():
    if len(indices) > 1:
        table = result.loc[indices]
        plt.figure().set_figheight(10)
        plt.figure().set_figwidth(10)
        plt.scatter(table.index, table['runAvg'])
        plt.scatter(table.index, table['runMdn'])
        plt.scatter(table.index, table['runStD'])
        plt.xticks(rotation=45)
        plt.xlabel(key)  # Etichetta sull'asse x
        plt.ylabel('Value')  # Etichetta sull'asse y
        plt.show()
       # print(f"Group: {key}")
        #print(result.loc[indices])
        #print("---")
